# Исследование для сервиса для чтения книг по подписке.

В нашем распоряжении база данных, в ней —  информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Наша задача — проанализиоровать базу данных о книгах**

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Исследование будет проходить в пять этапов:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Подлючимся к БД и откроем файлы с данными

**Подключение**

In [1]:
# — тире для себя
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': '****', # имя пользователя
'pwd': '****;****', # пароль
'host': '****',
'port': ****, # порт подключения
'db': '****'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Выгрузим таблицу books**

In [2]:
query = '''
SELECT *
FROM books
''' 

In [3]:
books = pd.read_sql_query(query, con = engine)

In [4]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Выгрузим таблицу authors**

In [5]:
query = '''
SELECT *
FROM authors
''' 

In [6]:
authors = pd.read_sql_query(query, con = engine)

In [7]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Выгрузим таблицу publishers**

In [8]:
query = '''
SELECT *
FROM publishers
''' 

publishers = pd.read_sql_query(query, con = engine)

In [9]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Выгрузим таблицу ratings**

In [10]:
query = '''
SELECT *
FROM ratings
''' 

ratings = pd.read_sql_query(query, con = engine)

In [11]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Выгрузим таблицу reviews**

In [12]:
query = '''
SELECT *
FROM reviews
''' 

reviews = pd.read_sql_query(query, con = engine)

In [13]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Ознакомление с таблицами прошло успешно**

### Исследование

**Посчитаем, сколько книг вышло после 1 января 2000 года**

In [14]:
query = '''
SELECT 
    COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
''' 

count_books = pd.read_sql_query(query, con = engine)

In [15]:
count_books

,count
0,819


***Вывод: После 1 января 2000 года вышло 819 книг***

**Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [16]:
query = '''
SELECT 
    b.book_id,
    b.title,
COUNT(DISTINCT(rev.review_id)) AS count,
AVG(rat.rating) AS avg_rat
FROM books AS b
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY avg_rat DESC, count DESC
''' 

count_books = pd.read_sql_query(query, con = engine)

In [17]:
count_books

,book_id,title,count,avg_rat
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


***Вывод: Самый высокий рейтинг(5.0) и кол-во обзоров(4) имеет книга A Dirty Job (Grim Reaper #1)***

**Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**

In [18]:
query = '''

WITH 
num AS 
(
SELECT 
    publisher_id,
    title
FROM books
GROUP BY book_id
HAVING num_pages  > 50
)

SELECT 
    DISTINCT publisher, 
    COUNT(pub.title) AS cnt_book_id
FROM num AS pub
INNER JOIN publishers p ON p.publisher_id = pub.publisher_id
GROUP BY publisher
ORDER BY cnt_book_id DESC
LIMIT 1
''' 

top_publishers = pd.read_sql_query(query, con = engine)

In [19]:
top_publishers

,publisher,cnt_book_id
0,Penguin Books,42


***Вывод: Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books (42 книги)***

**Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками**

In [20]:
query = '''


SELECT 
    DISTINCT a.author AS aut, 
    ROUND(AVG(r.rating), 3) as top_avg
FROM authors AS a
INNER JOIN books AS b ON b.author_id = a.author_id
INNER JOIN ratings AS r ON r.book_id = b.book_id
WHERE b.book_id in 
(
SELECT 
    book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating) >= 50
)
GROUP BY author
ORDER BY top_avg DESC
LIMIT 1
''' 

top_author = pd.read_sql_query(query, con = engine)

In [21]:
top_author

,aut,top_avg
0,J.K. Rowling/Mary GrandPré,4.287


***Вывод: Автор с самой высокой средней оценкой книг, с учетом книг с 50 и более оценками — J.K. Rowling/Mary GrandPré***

**Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [38]:
query = '''
WITH x AS 
(SELECT 
    username,
    COUNT(review_id) as count
FROM reviews
GROUP BY username
HAVING username IN (SELECT 
                         username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50))
                    
SELECT
    ROUND(AVG(count))
FROM x
        '''

avg_review = pd.io.sql.read_sql(query, con = engine)

In [39]:
avg_review

,round
0,24.0


**Среднее количество обзоров — 24**

### Общий вывод

После проведения исследования, можно сделать вывод, что:
- После 1 января 2000 года вышло — 819 книг;
- Самый высокий рейтинг(5.0) и кол-во обзоров(4) имеет книга — A Dirty Job (Grim Reaper #1);
- Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books (42 книги);
- Автор с самой высокой средней оценкой книг, с учетом книг с 50 и более оценками — J.K. Rowling/Mary GrandPré;
- Среднее количество обзоров — 24.